4. MODEL DEPLOYMENT

We have a module for getting and storing our data. We have the code to train our model and clean its predictions. We put all this pieces  together and deploy our model with an API that others can use to train their own models and predict volatility. We'll start by creating a model for all the code we created in the last lesson. Then we'll complete our main module, which will hold our FastAPI application with two paths: one for model training and one for prediction.


In [16]:
%load_ext autoreload
%autoreload 2

import os
import sqlite3
from glob import glob
import warnings

import joblib
import pandas as pd
import requests
from arch.univariate.base import ARCHModelResult
from config import settings
from data import SQLRepository
warnings.simplefilter(action="ignore", category=FutureWarning)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


4.1 Model Module

We will create a model module to make our code  reusable
Start by instantiating a repository to be used for testing our module as we build.

In [17]:
#Create a SQLRepository named repo and attach it to a SQLite connection.
connection = sqlite3.connect(settings.db_name, check_same_thread=False)
repo = SQLRepository(connection=connection)

print("repo type:", type(repo))
print("repo.connection type:", type(repo.connection))

repo type: <class 'data.SQLRepository'>
repo.connection type: <class 'sqlite3.Connection'>


In the model module, create a definition for a GarchModel model class. Use __init__ method. 

In [18]:
#Test your class using the assert statements
from model import GarchModel

# Instantiate a `GarchModel`
gm_cic = GarchModel(ticker="CIC", repo=repo, use_new_data=False)

# Does `gm_ambuja` have the correct attributes?
assert gm_cic.ticker == "CIC"
assert gm_cic.repo == repo
assert not gm_cic.use_new_data
assert gm_cic.model_directory == settings.model_directory

Turn your wrangle_data function a method for the GarchModel class.


In [19]:
#Use the assert statements below to test the method by getting and wrangling data for Co-operative  Bank

# Instantiate `GarchModel`, use new data
model_coop = GarchModel(ticker="COOP", repo=repo, use_new_data=True)

# Check that model doesn't have `data` attribute yet
assert not hasattr(model_coop, "data")

# Wrangle data
model_coop.wrangle_data(n_observations=1300)

# Does model now have `data` attribute?
assert hasattr(model_coop, "data")

# Is the `data` a Series?
assert isinstance(model_coop.data, pd.Series)

# Is Series correct shape?
assert model_coop.data.shape == (1300,)

model_coop.data.head()

date
2017-12-28    3.458065
2017-12-29    5.899227
2018-01-02    0.482864
2018-01-03   -2.461322
2018-01-04   -2.162942
Name: return, dtype: float64

Use the previous code for creating garch model to create a fit method for the GarchModel class

In [20]:
#Use the assert statements below to test the method

# Instantiate `GarchModel`, use old data
model_coop = GarchModel(ticker="COOP", repo=repo, use_new_data=False)

# Wrangle data
model_coop.wrangle_data(n_observations=1000)

# Fit GARCH(1,1) model to data
model_coop.fit(p=1, q=1)

# Does `model_shop` have a `model` attribute now?
assert hasattr(model_coop, "model")

# Is model correct data type?
assert isinstance(model_coop.model, ARCHModelResult)

# Does model have correct parameters?
assert model_coop.model.params.index.tolist() == ["mu", "omega", "alpha[1]", "beta[1]"]

# Check model parameters
model_coop.model.summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2419.21
Distribution:                  Normal   AIC:                           4846.42
Method:            Maximum Likelihood   BIC:                           4866.05
                                        No. Observations:                 1000
Date:                Wed, Mar 01 2023   Df Residuals:                      999
Time:                        15:22:38   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.1646  8.049e-02      2.045  4.089e-02 [6.818e-03,  0.322]
                             Volatility Model                             
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
omega          0.2077      0.166      1.254      0.210   [ -0.117,  0.532]
alpha[1]       0.1045  4.677e-02      2.234  2.545e-02 [1.284e-02,  0.196]
beta[1]        0.8771  5.123e-02     17.120  1.059e-65   [  0.777,  0.977]
==========================================================================

Covariance estimator: robust
"""

Create a predict_volatility method for your GarchModel class using the code from the garch-models module. The method will return predictions as a dictionary, therefore clean_prediction function will be added as a helper method. 

In [21]:
# Generate prediction from `model_shop`
prediction = model_coop.predict_volatility(horizon=5)

# Is prediction a dictionary?
assert isinstance(prediction, dict)

# Are keys correct data type?
assert all(isinstance(k, str) for k in prediction.keys())

# Are values correct data type?
assert all(isinstance(v, float) for v in prediction.values())

prediction

{'2023-03-01T00:00:00': 1.9633575128960912,
 '2023-03-02T00:00:00': 1.997867670868723,
 '2023-03-03T00:00:00': 2.031171641285121,
 '2023-03-06T00:00:00': 2.0633390844398547,
 '2023-03-07T00:00:00': 2.0944332487026647}

Add two more methods to the GarchModel to save a trained model and load it up when needed

Use the joblib library and write the filepaths using the os library.

In [23]:
ticker = "SCOM"
pattern = os.path.join(settings.model_directory, f"*{ticker}.pkl")

try:
    model_path=sorted(glob(pattern))[-1]
except IndexError:
    raise Exception(f"No model trained for '{ticker}'.")

Exception: No model trained for 'SCOM'.